In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dataOriDf = pd.read_excel('../../data/데이터정리3.xlsx')

pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)

#1977년 합격자부터 2019년 합격자
dataDf = dataOriDf[['년도','의대졸합격/남', '의대졸합격/여', '의전졸합격/남', '의전졸합격/여', '불합합격/남', '불합합격/여']]
dataDf = dataDf[27:70]

npData = np.around(np.array(dataDf)[:,1:7])

# 분류모델함수 생성
# x - x값, sp - 시작점, xl - x축 조정, yl - y축 조정
def sigmoid_function(x, sp, xl, yl):
    
    result = 1 / ( yl+np.exp((x-(sp+(10/xl/2)))*xl) )
    
    for i in range(np.size(x)):
        if x[i]<sp:
           result[i]=0
        if x[i]>(sp+(10/xl)):
           result[i]=0
            
    return result

# 분류모델 확률질량함수 생성

def sigmoid_model(sp, xl, yl):
    x = np.arange(1, 101, 1)
    y = sigmoid_function(x, sp, xl, yl)
    sum = np.sum(y)
    z = y/sum
    
    return z

def makeArrayUseModel(xlList, ylList):
    model1 = sigmoid_model(26,xlList[0],ylList[0]) # 의대 남
    model2 = sigmoid_model(26,xlList[1],ylList[1]) # 의대 여
    model3 = sigmoid_model(28,xlList[2],ylList[2]) # 의전원 남
    model4 = sigmoid_model(28,xlList[3],ylList[3]) # 의전원 여
    model5 = sigmoid_model(27,xlList[4],ylList[4]) # 재시험 남
    model6 = sigmoid_model(27,xlList[5],ylList[5]) # 재시험 여
    result = [model1, model2, model3, model4, model5, model6]
    
    return np.array(result)

def shiftOld(personArray):
    dataArray = np.zeros((2,len(personArray[0])))
    dataArray[0] = np.roll(personArray[0], 1)  
    dataArray[1] = np.roll(personArray[1], 1)
    dataArray[0][0] = 0
    dataArray[1][0] = 0
    return dataArray


In [110]:
xlList = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
ylList = [0.005, 0.005, 0.005, 0.005, 0.005, 0.005]

modelAry = makeArrayUseModel(xlList, ylList)

yearSize = len(npData)
oldSize = 100
modelSize = len(modelAry)

#모델 적용 배열
applyModelArray = np.zeros((yearSize,modelSize,oldSize))

#신규인원 배열
newPersonArray = np.zeros((yearSize,2,oldSize))

#누적인원 배열
personArray = np.zeros((yearSize,2,oldSize))

for i in range(yearSize):
    for j in range(modelSize):
        applyModelArray[i][j] = np.around(modelAry[j]*np.around(npData[i][j]))
        
        if j%2==0:
            newPersonArray[i][0] += applyModelArray[i][j]
        else:
            newPersonArray[i][1] += applyModelArray[i][j]



for i in range(yearSize):
    if i>0:
        personArray[i] = shiftOld(personArray[i-1]) + newPersonArray[i]
    else:
        personArray[i] =  newPersonArray[i]